Load API Keys

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

Load Contents from Brillar Bank

In [2]:
from langchain_community.document_loaders import WebBaseLoader
from helpers.test_data import get_urls

loader = WebBaseLoader(get_urls())

data = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


Split the Loaded Data into Chunks

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=20)
texts = text_splitter.split_documents(data)

Get Embedding Model

In [4]:
from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(model="text-embedding-3-large", dimensions=256)

Instantiate Elasticsearch Store

In [5]:
from langchain_elasticsearch import ElasticsearchStore, DenseVectorStrategy
import os

es_cloud_id = os.getenv("ELASTIC_CLOUD_ID")
es_api_key = os.getenv("ELASTIC_API_KEY")

dense_vector_store = ElasticsearchStore.from_documents(
    documents=texts,
    es_cloud_id=es_cloud_id,
    es_api_key=es_api_key,
    index_name="just_dense_vector",
    embedding=embedding,
    strategy=DenseVectorStrategy()
)

Get LLM

In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

Get Retriever

In [7]:
retriever = dense_vector_store.as_retriever(search_kwargs={"k": 5})

Create Chain

In [8]:
from helpers.conversation_retrieval_chain import create_conversational_retrieval_chain

chain = create_conversational_retrieval_chain(llm, retriever)

Invoke Chain

In [9]:
from helpers.conversation_retrieval_chain import invoke_conversational_retrieval_chain
from helpers.test_data import get_questions

questions = get_questions()
for question in questions:
  result = invoke_conversational_retrieval_chain(chain, question)
  print(f'{question}\n{result["answer"]}\n')

How many types of fixed deposit does Hong Leong Bank provide?
Hong Leong Bank provides six types of fixed deposit accounts: Fixed Deposit, e-Fixed Deposit, Flexi Fixed Deposit, Senior Savers Flexi Fixed Deposit, Junior Fixed Deposit, and Foreign Currency Fixed Deposit.

What are the interest rates for fixed deposit?
The interest rates for fixed deposit accounts are as follows:
- 1 month: 2.15% p.a.
- 2-3 months: 2.25% p.a.
- 4-5 months: 2.30% p.a.
- 6 months: 2.30% p.a.
- 7-11 months: 2.35% p.a.
- 12-60 months: 2.50% p.a.

What are the interest rates for e-fixed deposit?
The interest rates for e-fixed deposits at Hong Leong Bank are as follows:

- 1 month: 1.85% p.a.
- 2 months: 2.25% p.a.
- 3 months: 2.25% p.a.
- 4-6 months: 2.30% p.a.
- 7-12 months: 2.35% p.a.
- 13-60 months: 2.35% p.a.

What are the interest rates for flexi-fixed deposit?
The interest rate for the Flexi Fixed Deposit with a 12-month tenure is 2.50% per annum.

What are the interest rates for junior fixed deposit?
Th